In [19]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
import statsmodels.api as sm
import numpy as np
from itertools import product

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]


In [20]:
fees_df = dt.get_territory_post_fee_histories()

posts_df = dt.get_posts()
posts_df['subName'] = posts_df['subName'].fillna('')
posts_df['subName'] = posts_df['subName'].astype(str)

billing_cycles = dt.get_territory_billing_cycles()
billing_cycles['billing_cycle_start'] = billing_cycles['billing_cycle_start'].dt.floor('D')
billing_cycles['billing_cycle_end'] = billing_cycles['billing_cycle_end'].dt.floor('D')
billing_cycles['subName'] = billing_cycles['subName'].fillna('')
billing_cycles['subName'] = billing_cycles['subName'].astype(str)

In [21]:
globals.data_start.floor('D')

Timestamp('2021-06-11 00:00:00+0000', tz='UTC')

In [22]:
# Initialize territory-daily panel

subs = list(posts_df['subName'].unique())
N_days = (globals.data_end.date() - globals.data_start.date()).days
dates = [(globals.data_start.date() + pd.DateOffset(days=i)).floor('D') for i in range(N_days+1)]
tdf = pd.DataFrame(list(product(subs, dates)), columns=['subName', 'date'])
tdf['date'] = tdf['date'].dt.tz_localize('UTC')

In [23]:
# first post date for each sub
# drop dates before first post date

first_post_dates = posts_df.groupby('subName').agg(
    first_post_date = ('created_at', 'min')
).reset_index()
first_post_dates['first_post_date'] = first_post_dates['first_post_date'].dt.floor('D')

tdf = tdf.merge(first_post_dates, on='subName', how='left')
tdf = tdf[tdf['date'] >= tdf['first_post_date']].reset_index(drop=True)
tdf = tdf.drop(columns=['first_post_date'])

In [24]:
tdf['date'].iloc[0]

Timestamp('2021-06-11 00:00:00+0000', tz='UTC')

In [25]:
# merge on billing cycles

tdf = tdf.sort_values(by='date', ascending=True)
billing_cycles = billing_cycles.sort_values(by='billing_cycle_start', ascending=True)

tdf = pd.merge_asof(
    tdf,
    billing_cycles[['subName', 'billing_cycle_start', 'billing_cycle_end']],
    by='subName',
    left_on='date',
    right_on='billing_cycle_start',
    direction='backward'
)


In [26]:
# merge on n_posts

mask = posts_df['invoiceActionState'] != 'FAILED'
posts_df['date'] = posts_df['created_at'].dt.floor('D')

n_posts = posts_df.loc[mask].groupby(['subName', 'date']).agg(
    n_posts = ('itemId', 'count')
).reset_index()

tdf = tdf.merge(n_posts, on=['subName', 'date'], how='left')
tdf['n_posts'] = tdf['n_posts'].fillna(0)

In [53]:
bad = (tdf['date'] > tdf['billing_cycle_end']) & (tdf['n_posts']>0)
tdf.loc[bad].to_csv('temp.csv', index=False)

In [67]:
idf = pd.read_parquet(os.path.join(RAW_DATA_PATH, "invoice.parquet"))
mask = (idf['actionState'] == 'PAID') & (idf['actionType'].isin(['TERRITORY_CREATE', 'TERRITORY_UNARCHIVE']))
idf = idf.loc[mask].reset_index(drop=True)
idf['sats'] = idf['msatsReceived'] / 1000

In [68]:
idf[['userId', 'created_at', 'sats', 'actionState', 'actionType']]

,userId,created_at,sats,actionState,actionType
0,4166,2024-09-14 14:34:17.005,100000.0,PAID,TERRITORY_CREATE
1,18528,2025-01-02 12:00:56.158,50000.0,PAID,TERRITORY_CREATE
2,29006,2025-05-04 21:07:43.410,50000.0,PAID,TERRITORY_UNARCHIVE
3,27940,2025-06-23 19:22:32.846,50000.0,PAID,TERRITORY_UNARCHIVE
4,18934,2024-08-09 20:34:37.267,100000.0,PAID,TERRITORY_CREATE
5,5173,2024-07-03 11:30:06.232,100000.0,PAID,TERRITORY_UNARCHIVE
6,985,2024-10-18 23:19:58.498,1000000.0,PAID,TERRITORY_UNARCHIVE
7,1983,2024-09-19 22:43:37.051,100000.0,PAID,TERRITORY_CREATE
8,985,2024-10-19 18:16:34.652,1000000.0,PAID,TERRITORY_UNARCHIVE
9,21296,2024-09-23 07:18:12.970,3000000.0,PAID,TERRITORY_UNARCHIVE


In [71]:
billing_cycles.loc[billing_cycles['subName']=='mempool']

,subName,userId,billing_cycle_start,billing_cycle_end,sats,period
316,mempool,3656,2024-01-02 00:00:00+00:00,2024-02-07 00:00:00+00:00,100000.0,MONTHLY


In [70]:
billing_cycles.loc[billing_cycles['userId']==18528]

,subName,userId,billing_cycle_start,billing_cycle_end,sats,period
69,based,18528,2024-10-20 00:00:00+00:00,2024-11-25 00:00:00+00:00,100000.0,MONTHLY
94,bitcoin_support,18528,2025-01-02 00:00:00+00:00,2025-02-07 00:00:00+00:00,50000.0,MONTHLY


In [65]:
idf.loc[idf['userId']==17106]

,id,created_at,updated_at,userId,hash,bolt11,expiresAt,confirmedAt,cancelled,msatsRequested,...,actionArgs,actionError,actionResult,actionOptimistic,cancelledAt,predecessorId,userCancel,paymentAttempt,retryPendingSince,sats


In [56]:
posts_df.loc[posts_df['itemId']==410922]

,itemId,created_at,updated_at,title,text,url,userId,parentId,path,pinId,...,subWeightedDownVotes,subWeightedVotes,randPollOptions,saloon,root_subName,root_is_saloon,n_uploads,hasImageOrLink,cost_modifier,date
95180,410922,2024-02-02 21:44:53.712000+00:00,2024-02-02 21:54:55.027,Akua Kelp Burgers,So I invested in this kelp food startup via Re...,None,17106,NaN,410922,NaN,...,0.0,0.0,False,False,None,None,1,True,0,2024-02-02 00:00:00+00:00
